<a href="https://colab.research.google.com/github/bburt26/Final-Project/blob/main/milestone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone in Fork

In [1]:
! git clone https://github.com/bburt26/Final-Project.git

Cloning into 'Final-Project'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 118 (delta 47), reused 43 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 1.07 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [ ]:
!git config --global user.name "bburt26"
!git config --global user.email "jtg3ts@virginia.edu"

In [11]:
%cd Final-Project

/content/Final-Project


In [ ]:
#! git push origin main

fatal: could not read Username for 'https://github.com': No such device or address


# AI Acknowledgement

I acknowledge that I have used AI to the extent permitted by the UVA AI acceptable use policy. I am aware of the possibility of AI producing errors and offensive content and certify that I have taken the necessary measures to ensure that the work produced is both accurate and ethical to the best of my ability.

# Cleaning and Wrangling

In [34]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/Final-Project/data/CVD Dataset.csv')

### --- Cleaning --- ###
columns_of_interest = ['Sex',	'Age',	'Weight (kg)',	'Height (m)',
                       'BMI',	'Total Cholesterol (mg/dL)',
                       'Fasting Blood Sugar (mg/dL)',
                       'Smoking Status',	'Diabetes Status',
                       'Physical Activity Level'	,
                       'Family History of CVD',	'CVD Risk Level',
                       'Systolic BP',	'Diastolic BP',	'Blood Pressure Category',
                      'CVD Risk Score'
                       ]
data_cleaned = data[columns_of_interest].copy()

# Replace Nulls with Mean
missing_numerical_cols = [
    'Age', 'Weight (kg)', 'Height (m)', 'BMI', 'Total Cholesterol (mg/dL)',
    'Fasting Blood Sugar (mg/dL)', 'Systolic BP', 'Diastolic BP', 'CVD Risk Score'
]

for col in missing_numerical_cols:
    if col in data_cleaned.columns:
        data_cleaned[col] = data_cleaned[col].fillna(data_cleaned[col].mean())


### --- Converting to Numeric --- ###
# Convert Age to int64
data_cleaned['Age'] = data_cleaned['Age'].astype('int64')

# Save cleaned dataframe to CSV
data_cleaned.to_csv('CVD_Cleaned.csv', index=False)


### --- Encoding Categorical Data --- ###
# Encode Sex as numeric
data_cleaned['Sex'] = data_cleaned['Sex'].map({'F': 0, 'M': 1})

# Encode BP Category as numeric
bp_map = {
    'Normal': 0,
    'Elevated': 1,
    'Hypertension Stage 1': 2,
    'Hypertension Stage 2': 3
}
data_cleaned['BP_Category_Label'] = data_cleaned['Blood Pressure Category'].map(bp_map)

# Encode Smoking Status
smoking_map = {'N': 0, 'Y': 1}
data_cleaned['Smoking_Status_Encoded'] = data_cleaned['Smoking Status'].map(smoking_map)

# Encode diabetes status
diabetes_map = {'N': 0, 'Y': 1}
data_cleaned['Diabetes_Status_Encoded'] = data_cleaned['Diabetes Status'].map(diabetes_map)

# Encode Physical Activity Level
physical_activity_map = {'Low': 0, 'Moderate': 1, 'High': 2}
data_cleaned['Physical_Activity_Level_Encoded'] = data_cleaned['Physical Activity Level'].map(physical_activity_map)

# Encode Family History of CVD
family_history_map = {'N': 0, 'Y': 1}
data_cleaned['Family_History_CVD_Encoded'] = data_cleaned['Family History of CVD'].map(family_history_map)

# Encode CVD Risk Level
cvd_risk_map = {'LOW': 0, 'INTERMEDIARY': 1, 'HIGH': 2}
data_cleaned['CVD_Risk_Level_Encoded'] = data_cleaned['CVD Risk Level'].map(cvd_risk_map)

print("Data types after encoding new columns:")
print(data_cleaned[['Smoking_Status_Encoded', 'Diabetes_Status_Encoded',
                    'Physical_Activity_Level_Encoded', 'Family_History_CVD_Encoded',
                    'CVD_Risk_Level_Encoded']].dtypes)

print("\nFirst 5 rows with new encoded columns:")
display(data_cleaned.head())

# Replace columns with encoded versions
columns_to_drop = [
    'Smoking Status',
    'Diabetes Status',
    'Physical Activity Level',
    'Family History of CVD',
    'CVD Risk Level',
    'Blood Pressure Category'
]

data_cleaned = data_cleaned.drop(columns=columns_to_drop)

# View
print("DataFrame after dropping original categorical columns:")
display(data_cleaned.head())
print("\nUpdated DataFrame Info:")
data_cleaned.info()

print("Missing values after imputation:")
print(data_cleaned[missing_numerical_cols].isnull().sum())
print("\nUpdated DataFrame Info:")
data_cleaned.info()


### ---- Save as modeling data --- ###
model_data = data_cleaned.copy()
model_data.head(10)


Data types after encoding new columns:
Smoking_Status_Encoded             int64
Diabetes_Status_Encoded            int64
Physical_Activity_Level_Encoded    int64
Family_History_CVD_Encoded         int64
CVD_Risk_Level_Encoded             int64
dtype: object

First 5 rows with new encoded columns:


,Sex,Age,Weight (kg),Height (m),BMI,Total Cholesterol (mg/dL),Fasting Blood Sugar (mg/dL),Smoking Status,Diabetes Status,Physical Activity Level,...,Systolic BP,Diastolic BP,Blood Pressure Category,CVD Risk Score,BP_Category_Label,Smoking_Status_Encoded,Diabetes_Status_Encoded,Physical_Activity_Level_Encoded,Family_History_CVD_Encoded,CVD_Risk_Level_Encoded
0,0,32,69.100000,1.71,23.6,248.0,111.0,N,Y,Low,...,125.0,79.0,Elevated,17.93,1,0,1,0,0,1
1,0,55,118.700000,1.69,41.6,162.0,135.0,Y,Y,High,...,139.0,70.0,Hypertension Stage 1,20.51,2,1,1,2,1,2
2,1,47,85.917427,1.83,26.9,103.0,114.0,N,N,High,...,104.0,77.0,Normal,12.64,0,0,0,2,1,1
3,1,44,108.300000,1.80,33.4,134.0,91.0,N,N,High,...,140.0,83.0,Hypertension Stage 1,16.36,2,0,0,2,1,1
4,0,32,99.500000,1.86,28.8,146.0,141.0,Y,Y,High,...,144.0,83.0,Hypertension Stage 1,17.88,2,1,1,2,0,1


DataFrame after dropping original categorical columns:


,Sex,Age,Weight (kg),Height (m),BMI,Total Cholesterol (mg/dL),Fasting Blood Sugar (mg/dL),Systolic BP,Diastolic BP,CVD Risk Score,BP_Category_Label,Smoking_Status_Encoded,Diabetes_Status_Encoded,Physical_Activity_Level_Encoded,Family_History_CVD_Encoded,CVD_Risk_Level_Encoded
0,0,32,69.100000,1.71,23.6,248.0,111.0,125.0,79.0,17.93,1,0,1,0,0,1
1,0,55,118.700000,1.69,41.6,162.0,135.0,139.0,70.0,20.51,2,1,1,2,1,2
2,1,47,85.917427,1.83,26.9,103.0,114.0,104.0,77.0,12.64,0,0,0,2,1,1
3,1,44,108.300000,1.80,33.4,134.0,91.0,140.0,83.0,16.36,2,0,0,2,1,1
4,0,32,99.500000,1.86,28.8,146.0,141.0,144.0,83.0,17.88,2,1,1,2,0,1



Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Sex                              1529 non-null   int64  
 1   Age                              1529 non-null   int64  
 2   Weight (kg)                      1529 non-null   float64
 3   Height (m)                       1529 non-null   float64
 4   BMI                              1529 non-null   float64
 5   Total Cholesterol (mg/dL)        1529 non-null   float64
 6   Fasting Blood Sugar (mg/dL)      1529 non-null   float64
 7   Systolic BP                      1529 non-null   float64
 8   Diastolic BP                     1529 non-null   float64
 9   CVD Risk Score                   1529 non-null   float64
 10  BP_Category_Label                1529 non-null   int64  
 11  Smoking_Status_Encoded           1529 non-null   int64  


,Sex,Age,Weight (kg),Height (m),BMI,Total Cholesterol (mg/dL),Fasting Blood Sugar (mg/dL),Systolic BP,Diastolic BP,CVD Risk Score,BP_Category_Label,Smoking_Status_Encoded,Diabetes_Status_Encoded,Physical_Activity_Level_Encoded,Family_History_CVD_Encoded,CVD_Risk_Level_Encoded
0,0,32,69.100000,1.71,23.6,248.0,111.0,125.000000,79.0,17.93,1,0,1,0,0,1
1,0,55,118.700000,1.69,41.6,162.0,135.0,139.000000,70.0,20.51,2,1,1,2,1,2
2,1,47,85.917427,1.83,26.9,103.0,114.0,104.000000,77.0,12.64,0,0,0,2,1,1
3,1,44,108.300000,1.80,33.4,134.0,91.0,140.000000,83.0,16.36,2,0,0,2,1,1
4,0,32,99.500000,1.86,28.8,146.0,141.0,144.000000,83.0,17.88,2,1,1,2,0,1
5,0,58,117.900000,1.87,33.7,156.0,82.0,142.000000,90.0,16.96,3,1,0,1,1,2
6,1,33,117.900000,1.82,35.6,184.0,87.0,96.000000,63.0,17.60,0,0,1,0,0,1
7,1,55,100.100000,1.74,33.1,287.0,96.0,115.000000,73.0,18.11,0,1,0,1,0,2
8,0,37,110.100000,1.86,31.8,177.0,70.0,125.627572,95.0,15.25,3,1,0,2,0,1
9,0,58,97.600000,1.86,28.2,281.0,126.0,133.000000,83.0,17.91,2,0,0,1,0,1


# EDA

In [ ]:
# Code

# Modeling

In [ ]:
# code

# Confusion Metrics

In [ ]:
# Code

# Analysis

In [ ]:
# Text